# Concise Implementation of Linear Regression

Previously, we relied only on
(i) tensors for data storage and linear algebra;
and (ii) auto differentiation for calculating gradients.
In practice, because data iterators, loss functions, optimizers,
and neural network layers
are so common, modern libraries implement these components for us as well.

In this section, **we will see how to implement
the linear regression model** by deep learning frameworks.


## Generating the Dataset

To start, we will generate the same dataset as in :numref:`sec_linear_scratch`.


In [12]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [13]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## Reading the Dataset

We pass in `features` and `labels` as arguments and specify `batch_size`
when instantiating a data iterator object.
Besides, the boolean value `is_train`
indicates whether or not
we want the data iterator object to shuffle the data
on each epoch (pass through the dataset).


In [14]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [15]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

Now we can use `data_iter` in much the same way as we called
the `data_iter` function before.
To verify that it is working, we can read and print
the first minibatch of examples.
Here we use `iter` to construct a Python iterator and use `next` to obtain the first item from the iterator.


In [16]:
next(iter(data_iter))

[tensor([[ 0.8457,  1.5058],
         [-0.4979, -0.4224],
         [-0.0700,  1.1151],
         [ 0.9337,  0.0052],
         [-0.1477,  0.1838],
         [-0.1405,  0.0309],
         [-0.6126,  0.0162],
         [ 0.8446,  1.2903],
         [ 0.2773,  0.2839],
         [-1.6473,  0.7124]]),
 tensor([[ 0.7725],
         [ 4.6236],
         [ 0.2603],
         [ 6.0519],
         [ 3.2657],
         [ 3.8232],
         [ 2.9165],
         [ 1.5031],
         [ 3.8010],
         [-1.5302]])]

## Defining the Model

For standard operations, we can **use a framework's predefined layers,**
which allow us to focus especially
on the layers used to construct the model
rather than having to focus on the implementation.

We will first define a model variable `net`,
which will refer to an instance of the `Sequential` class.
The `Sequential` class defines a container
for several layers that will be chained together.
Given input data, a `Sequential` instance passes it through
the first layer, in turn passing the output
as the second layer's input and so forth.

In the following example, our model consists of only one layer,
so we do not really need `Sequential`.
But since nearly all of our future models
will involve multiple layers,
we will use it anyway just to familiarize you
with the most standard workflow.

The layer is said to be *fully-connected*
because each of its inputs is connected to each of its outputs
by means of a matrix-vector multiplication.


In PyTorch, the fully-connected layer is defined in the `Linear` class. Note that we passed two arguments into `nn.Linear`. The first one specifies the input feature dimension, which is 2, and the second one is the output feature dimension, which is a single scalar and therefore 1.


In [19]:
# `nn` is an abbreviation for neural networks
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

## Initializing Model Parameters

Before using `net`, we need to **initialize the model parameters,**
such as the weights and bias in the linear regression model.
Deep learning frameworks often have a predefined way to initialize the parameters.
Here we specify that each weight parameter
should be randomly sampled from a normal distribution
with mean 0 and standard deviation 0.01.
The bias parameter will be initialized to zero.


As we have specified the input and output dimensions when constructing `nn.Linear`,
now we can access the parameters directly to specify their initial values.
We first locate the layer by `net[0]`, which is the first layer in the network,
and then use the `weight.data` and `bias.data` methods to access the parameters.
Next we use the replace methods `normal_` and `fill_` to overwrite parameter values.


In [18]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## Defining the Loss Function


**The `MSELoss` class computes the mean squared error without the $1/2$ factor.**
By default it returns the average loss over examples.


In [17]:
loss = nn.MSELoss()

## Defining the Optimization Algorithm


Minibatch stochastic gradient descent is a standard tool
for optimizing neural networks
and thus PyTorch supports it alongside a number of
variations on this algorithm in the `optim` module.
When we **instantiate an `SGD` instance,**
we will specify the parameters to optimize over
(obtainable from our net via `net.parameters()`), with a dictionary of hyperparameters
required by our optimization algorithm.
Minibatch stochastic gradient descent just requires that
we set the value `lr`, which is set to 0.03 here.


In [20]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## Training

You might have noticed that expressing our model through
high-level APIs of a deep learning framework
requires comparatively few lines of code.
We did not have to individually allocate parameters,
define our loss function, or implement minibatch stochastic gradient descent.
Once we start working with much more complex models,
advantages of high-level APIs will grow considerably.
However, once we have all the basic pieces in place,
**the training loop itself is strikingly similar
to what we did when implementing everything from scratch.**

To refresh your memory: for some number of epochs,
we will make a complete pass over the dataset (`train_data`),
iteratively grabbing one minibatch of inputs
and the corresponding ground-truth labels.
For each minibatch, we go through the following ritual:

* Generate predictions by calling `net(X)` and calculate the loss `l` (the forward propagation).
* Calculate gradients by running the backpropagation.
* Update the model parameters by invoking our optimizer.

For good measure, we compute the loss after each epoch and print it to monitor progress.


In [21]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000320
epoch 2, loss 0.000101
epoch 3, loss 0.000101


Below, we **compare the model parameters learned by training on finite data
and the actual parameters** that generated our dataset.
To access parameters,
we first access the layer that we need from `net`
and then access that layer's weights and bias.
As in our from-scratch implementation,
note that our estimated parameters are
close to their ground-truth counterparts.


In [22]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([0.0002, 0.0006])
error in estimating b: tensor([-0.0003])
